One of the most important portion of the effort behind building a new transformer model is creating the new model tokenizer. The tokenizer is our translator from human-readable text, to transformer readable tokens. In this article, we will learn exactly how to build our own transformer tokenizer.

In this tutorial we will use the multi-lingual OSCAR dataset from huggingface to train a tokenizer.

In [ ]:
# !pip install datasets
# !pip install tokenizers

### Prepare dataset

In [1]:
import datasets

In [2]:
all_ds = datasets.list_datasets()
print("total number of datasets", len(all_ds))

total number of datasets 56612


In [3]:
all_ds[:10]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus',
 'ag_news',
 'ai2_arc',
 'air_dialogue',
 'ajgt_twitter_ar',
 'allegro_reviews']

In [4]:
# select the OSCAR dataset
dataset = datasets.load_dataset('oscar', 'unshuffled_deduplicated_la')


Found cached dataset oscar (/Users/ndah/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_la/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 18808
    })
})

In [6]:
# First record
# From here we can see that the Latin subset contains 18.8K samples, where each sample is a dictionary containing an id and text.
dataset['train'][0]

{'id': 0,
 'text': 'Hæ sunt generationes Noë: Noë vir justus atque perfectus fuit in generationibus suis; cum Deo ambulavit.\nEcce ego adducam aquas diluvii super terram, ut interficiam omnem carnem, in qua spiritus vitæ est subter cælum: universa quæ in terra sunt, consumentur.\nTolles igitur tecum ex omnibus escis, quæ mandi possunt, et comportabis apud te: et erunt tam tibi, quam illis in cibum.'}

We will store all of our samples in plain text files, separating each sample by a newline character.
We will split each text file into chunks of 5K samples each (although not necessary with a dataset of this size — it’s required for large datasets) and save them into a new oscar_la directory.

In [7]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'data/oscar_la/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# after saving in 5K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'data/oscar_la/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/18808 [00:00<?, ?it/s]

### Train a Latin roBERTa Tokenizer

To train the tokenizer we make use of the Byte pair encoding (not specific to NLP - its a compression algorithm)

Byte-level encoding means we will be building our tokenizer vocabulary from an alphabet of bytes. Thanks to this, all words will be decomposable into tokens — even new words — and so we will not need special unknown tokens.

We need a list of files to feed into our tokenizer’s training process, we will list all .txt files from our oscar_la directory.

In [8]:
# list files
from pathlib import Path
paths = [str(x) for x in Path('data/oscar_la').glob('**/*.txt')]
paths

['data/oscar_la/text_2.txt',
 'data/oscar_la/text_3.txt',
 'data/oscar_la/text_1.txt',
 'data/oscar_la/text_0.txt']

We use roBERTa special tokens, a vocabulary size of 30522 tokens, and a minimum frequency (number of times a token appears in the data for us to take notice) of 2

In [9]:
from tokenizers import ByteLevelBPETokenizer
import os

# initialize
tokenizer = ByteLevelBPETokenizer()

# and train
tokenizer.train(files=paths, 
                vocab_size=30_522, 
                min_frequency=2, # number of times a token appears in the data
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [11]:
# save tokenizer
tk_path = 'models/bertius'
os.mkdir(tk_path)

tokenizer.save_model(tk_path)

### Using the Tokenizer

In [12]:
from transformers import RobertaTokenizerFast

tk_path = 'models/bertius'
tokenizer = RobertaTokenizerFast.from_pretrained(tk_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [14]:
# example
lorem_ipsum = (
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor "
    "incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud "
    "exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute "
    "irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla "
    "pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia "
    "deserunt mollit anim id est laborum."
)

lorem_ipsum = (
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor "
    "incididunt ut labore et dolore magna aliqua."
)

In [15]:
# we'll include the typical padding/truncation
lorem_tk = tokenizer(lorem_ipsum, 
                     max_length=512, 
                     padding='max_length', 
                     truncation=True)


In [16]:
lorem_tk.keys()

dict_keys(['input_ids', 'attention_mask'])

Here we can see our two tensors, input_ids and attention_mask. In input_ids we can see our start of sequence token __<*s*>__ represented by 0, the end of sequence token <s\\> represented by 2, and padding tokens <*pad*> represented by 1.

In [17]:
lorem_tk

{'input_ids': [0, 3586, 652, 1600, 460, 1787, 16, 2617, 3713, 3087, 16, 397, 3701, 13753, 3726, 16098, 329, 2218, 290, 1913, 1546, 1649, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Training a transformer from scratch
1. [Implementing the Transformer Encoder from Scratch in TensorFlow and Keras](https://machinelearningmastery.com/implementing-the-transformer-encoder-from-scratch-in-tensorflow-and-keras/)
2. [Implementing the Transformer Decoder from Scratch in TensorFlow and Keras](https://machinelearningmastery.com/implementing-the-transformer-decoder-from-scratch-in-tensorflow-and-keras/)
3. [Training the Transformer Model](https://machinelearningmastery.com/training-the-transformer-model/)
4. [Building Transformer Models with Attention](https://machinelearningmastery.com/building-transformer-models-with-attention-crash-course-build-a-neural-machine-translator-in-12-days/)
5. [Training Compact Transformers from Scratch in 30 Minutes with PyTorch](https://medium.com/pytorch/training-compact-transformers-from-scratch-in-30-minutes-with-pytorch-ff5c21668ed5)

# Assigment 2 
### Due: August 27, 2023
In this assignment, you will construct a language models for any of the African language from [CC-100](https://data.statmt.org/cc-100/) using the huggginface platform. Provide the following information
1. Ensure you follow the appropriate machine learning paradigm to train your model
2. define/us an appropriate metric to evaluate the performance of the language model
3. Perform error analysis and give reasons for the limitations of your model if any.

Data source: [CC-100: Monolingual Datasets from Web Crawl Data](https://data.statmt.org/cc-100/)
